# Day 2 - Data Science Essentils Program

Agenda:
- Wrapping up pandas examples
    - pivot tables
    - advanced data cleaning and calcs
- Visualizations with pandas, matplotlib, seaborn, plotly
- Linear regressions

## Import Packages and Data

In [6]:
import pandas as pd
finData = pd.read_excel("ExData/Data Manipulation Worksheet.xlsx", sheet_name=1)

**Pivot Tables**
- groupby allows summary of categories on the "side" (rows or index)
- pivot_table you can summarize at top as well (columns)

In [9]:
pivot = pd.pivot_table(finData, values='SIZE', index=['INDUSTRY'], columns=['TYPE'],aggfunc=['sum','count'])
# pivot

In [10]:
pivot.to_excel('Output/pivot demo.xlsx')

## Advanced Pandas Functions
- calculations and data cleaning with `.apply()`
- when importing data check:
    - is there a better col to use as the index? 
    - dates should be datetime
    - numbers should be float or int:  `.astype(int)` or `pd.to_numeric()`
    - are there any missing values: `.dropna()` or `.fillna()`

In [12]:
ipoData = pd.read_csv('ExData/iposTable.csv')
ipoData.info()
ipoData.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Company            100 non-null    object 
 1   Symbol             100 non-null    object 
 2   Industry           100 non-null    object 
 3   Offer Date         100 non-null    object 
 4   Shares (millions)  100 non-null    float64
 5   Offer Price        100 non-null    object 
 6   1st Day Close      100 non-null    object 
 7   Current Price      100 non-null    object 
 8   Return             100 non-null    object 
 9   SCOOP Rating       100 non-null    object 
dtypes: float64(1), object(9)
memory usage: 7.9+ KB


,Company,Symbol,Industry,Offer Date,Shares (millions),Offer Price,1st Day Close,Current Price,Return,SCOOP Rating
0,Metals Acquisition Corp,MTAL.U,Blank Check,7/29/2021,25.0,$10.00,$0.00,$0.00,0.00%,S/O
1,"Nuvalent, Inc.",NUVL,Health Care,7/29/2021,9.8,$17.00,$0.00,$0.00,0.00%,S/O
2,Rallybio Corp.,RLYB,Health Care,7/29/2021,6.2,$13.00,$0.00,$0.00,0.00%,S/O
3,Software Acquisition Group Inc. III,SWAGU,Blank Check,7/29/2021,20.0,$10.00,$0.00,$0.00,0.00%,S/O
4,Traeger Inc. (TGPX Holdings I LLC),COOK,Industrials,7/29/2021,23.5,$18.00,$0.00,$0.00,0.00%,S/O


In [23]:
ipoData = pd.read_csv('ExData/iposTable.csv', index_col=['Symbol'], parse_dates=['Offer Date'] )
ipoData.info()

<class 'pandas.core.frame.DataFrame'>
Index: 100 entries, MTAL.U to LYEL
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Company            100 non-null    object        
 1   Industry           100 non-null    object        
 2   Offer Date         100 non-null    datetime64[ns]
 3   Shares (millions)  100 non-null    float64       
 4   Offer Price        100 non-null    object        
 5   1st Day Close      100 non-null    object        
 6   Current Price      100 non-null    object        
 7   Return             100 non-null    object        
 8   SCOOP Rating       100 non-null    object        
dtypes: datetime64[ns](1), float64(1), object(7)
memory usage: 7.8+ KB


In [17]:
# ipoData.loc['NUVL']['Offer Price']

'$17.00'

In [20]:
text = "$1,700.50"
float(text.strip("$").replace(",",""))

1700.5

In [24]:
ipoData['Offer Price'] = ipoData['Offer Price'].str.strip("$").str.replace(",","").astype(float)
ipoData['Current Price'] = pd.to_numeric(ipoData['Current Price'].str.strip("$").str.replace(",",""))

In [31]:
# ipoData.info()
# float(["100", "500","600"])
pd.to_numeric("500")
pd.to_numeric(["100", "500","600"]) #array of int
pd.to_numeric(["100.100", "500.2","600.50"]) #floats now

array([100.1, 500.2, 600.5])

In [34]:
ipoData = pd.read_csv('ExData/iposTable.csv', index_col=['Symbol'], parse_dates=['Offer Date'] )
colsToClean = ['Offer Price', 'Current Price', '1st Day Close']
for col in colsToClean:
    ipoData[col] = ipoData[col].str.strip("$").str.replace(",","").astype(float)

ipoData['Return'] = ipoData['Return'].str.strip("%").str.replace(",","").astype(float) / 100

## Cleaning with .apply()
- first create a function that cleans up one piece of text
- then use .apply() to apply that function on the entire column

In [37]:
sp500_data = pd.read_csv("ExData/sp500_Cos_ValMetrics.csv")
sp500_data.head()
    #market cap and EV need to clean up
    #B --> billion, T --> trillion

,Symbol,Market Cap (intraday),Enterprise Value,Trailing P/E,Forward P/E,PEG Ratio (5 yr expected),Price/Sales (ttm),Price/Book (mrq),Enterprise Value/Revenue,Enterprise Value/EBITDA,Beta
0,A,45.29B,46.9B,49.92,32.38,3.33,7.79,9.42,8.06,30.30,1.01
1,AAL,12.84B,47.76B,NaN,-286.00,0.05,1.00,NaN,3.72,-4.44,1.85
2,AAP,13.67B,16.37B,22.45,17.10,1.55,1.27,3.87,1.52,13.20,1.38
3,AAPL,2.43T,2.4T,32.74,27.27,1.53,7.47,35.13,7.38,24.05,1.21
4,ABBV,208.26B,280.2B,40.48,8.44,2.04,4.15,15.18,5.58,11.42,0.80


In [38]:
def cleanFinNumber(x):
    if 'B' in x: #keeps it as billions
        clean = float(x.strip("B").replace(",", ""))
    elif 'T' in x: #converts trillions to billions
        clean = float(x.strip("T").replace(",", "")) * 1000
    else: #in case some numbers don't have B and T, assume it's million --> convert billions
        clean = float(x.replace(',','')) / 1000
    return clean

In [47]:
#testing the function on one piece of text
cleanFinNumber("500.5T")

500500.0

In [48]:
sp500_data['EV (bn)'] = sp500_data['Enterprise Value'].apply(cleanFinNumber)
sp500_data['Mkt Cap (bn)'] = sp500_data['Market Cap (intraday)'].apply(cleanFinNumber)

In [52]:
#sp500_data[['EV (bn)','Enterprise Value']] #.info()

**.apply with more than 1 column**
- can apply a function on multiple columns, but need to specify the axis
    - axis = 1 --> going down, row by row
    - axis = 0 --> going across, col by col

In [53]:
import numpy as np
np.mean(sp500_data['EV (bn)'])

84.11954365079364

In [57]:
sp500 = pd.read_csv('StockData/SP500.csv', parse_dates=['Date'], index_col=['Date'])
sp500.apply(np.mean, axis=0) #taking average of each column
sp500[['Close','Open']].apply(np.mean, axis=1) #average of the prices, for each date/row

Date
2013-09-30    1684.405029
2013-10-01    1688.705017
2013-10-02    1692.885010
2013-10-03    1685.505005
2013-10-04    1684.645019
                 ...     
2018-09-24    2920.600097
2018-09-25    2918.655029
2018-09-26    2911.474976
2018-09-27    2912.824951
2018-09-28    2912.005004
Length: 1260, dtype: float64